In [2]:
import pandas as pd

In [11]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [41]:
raw = pd.read_csv('../data/raw/ld_clump_assoc.txt',delimiter='\t')
rename = {'CpG':'cpg',
 'Top SNP':'snp',
 'Beta': 'beta',
 'SE': 'se',
 'Cis/Trans':'cistrans',
 'CpG chr': 'cpg_chr',
 'CpG pos': 'cpg_pos',
 'SNP chr': 'snp_chr',
 'SNP pos': 'snp_pos',
 'P': 'pval'}

raw = data.rename(rename, axis=1)

lst = []
for i in raw['cistrans']:
    if i == 'cis':
        lst.append(True)
    else:
        lst.append(False)
raw['cistrans'] = lst
raw.to_csv('edit_ld_clump_assoc.csv')


In [112]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12','13']




In [2]:
# level-based network prototype 
import pandas as pd
from pathlib import Path
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)

In [7]:
# creating unique nodes for cpg and snp encoding level information 
# level 0 cpg --> query user
# level 1 snp --> cpg-snp
# level 1 cpg --> cpg-snp-cpg connection 

cpg_interest = ['cg13938959','cg23733394']

cpgs = []
snps = []
level = 2

for cpg in cpg_interest:
    
    start = 1
    cpgs.append({'id':cpg,
             'level':0})

    while start <= level:
        
        cpg_included = [cpg['id'] for cpg in cpgs if cpg['level'] < start]
        snps_to_include = data[data['cpg'].isin(cpg_included)]['snp'].values
        [snps.append({'id':snp,'level':start}) for snp in snps_to_include if snp not in [snp['id'] for snp in snps]]
        snps_included = [snp['id'] for snp in snps if snp['level'] == start]
        cpgs_to_include = data[data['snp'].isin(snps_included)]['cpg'].values
        [cpgs.append({'id':cpg,'level':start}) for cpg in cpgs_to_include if cpg not in [cpg['id'] for cpg in cpgs]] 
        start+= 1

edges = data[data['cpg'].isin([cpg['id'] for cpg in cpgs])]
edges = edges[edges['snp'].isin([snp['id'] for snp in snps])]

In [ ]:
data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0)


In [107]:
chromosome_distance = {'1':0, 
'2': 249250621,
'3': 492449994,
'4': 690472424,
'5': 881626700,
'6': 1062541960,
'7': 1233657027,
'8': 1392795690,
'9': 1539159712,
'10': 1680373143,
'11': 1815907890,
'12': 1950914406,
'13': 2084766301,
'14': 2199936179,
'15': 2307285719,
'16': 2409817111,
'17': 2500171864,
'18': 2581367074,
'19': 2659444322,
'20': 2718573305,
'21': 2781598825,
'22': 2829728720,
'X': 2881033286,
'Y': 3036303846}

minAssoc = 10
minDistance = 1e6
minPval = 1e-6
chromosomeList = ['1','2','3','4','5','6','7','8','9','10','11','12'] 
 

data = pd.read_csv('../data/edit_ld_clump_assoc.csv',index_col=0) 
annot = pd.read_csv('../data/cpg_annotation.csv',index_col=0)
chr_lst = chromosomeList
data['cpg_chr'] = data['cpg_chr'].apply(str)
data['snp_chr'] = data['snp_chr'].apply(str)

data = data[data['cistrans']==False]

data = data.groupby('cpg') # group data by cpgs
data = data.filter(lambda x: len(x) >= minAssoc) # filter by number of associations per cpg

data = data[data['cpg_chr'].isin(chr_lst)]
data = data[data['pval']<=1*10**-minPval]

data['cpg_pos_abs'] = data['cpg_chr'].apply(lambda cpg_chr: chromosome_distance[str(cpg_chr)]) + data['cpg_pos'] # calculate absolute distance of cpg
data['snp_pos_abs'] = data['snp_chr'].apply(lambda snp_chr: chromosome_distance[str(snp_chr)]) + data['snp_pos'] # calculate absolute distance of snp
data['dist'] = abs(data['cpg_pos_abs'] - data['snp_pos_abs']) # calculate distance between pairs
data = data[data['dist']>=minDistance] # filter by min distance

data = data.reset_index(drop=True)

data = data.merge(annot,how='left',on='cpg')
    

data.drop(['MAPINFO','CHR'],axis=1,inplace=True)

response = data.to_dict('records')  



In [109]:
data['UCSC_RefGene_Name']

0     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
1     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
2     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
3     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
4     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
5     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
6     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
7     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
8     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
9     DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
10    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
11    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
12    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
13    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
14    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
15    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
16    DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
Name: UCSC_RefGene_Name, dtype: object

In [110]:
data

,cpg,snp,LD clump,cpg_chr,cpg_pos,snp_chr,snp_pos,A1,A2,MAF,...,pval,FDR,N,n,Effects,cistrans,cpg_pos_abs,snp_pos_abs,dist,UCSC_RefGene_Name
0,cg15580874,6:30997692_G_C,6:30032522_A_C,2,179315077,6,30997692,G,C,0.328,...,1.290000e-17,4.890000e-08,2,1742,++???,False,428565698,1093539652,664973954,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
1,cg15580874,6:31187170_GT_G,6:31238009_T_C,2,179315077,6,31187170,GT,G,0.263,...,7.000000e-17,5.790000e-07,2,1742,++???,False,428565698,1093729130,665163432,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
2,cg15580874,6:31194764_A_C,6:31346599_G_A,2,179315077,6,31194764,A,C,0.212,...,3.480000e-20,0.000000e+00,2,1742,++???,False,428565698,1093736724,665171026,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
3,cg15580874,6:31482203_A_G,6:31461043_G_A,2,179315077,6,31482203,A,G,0.103,...,8.310000e-45,0.000000e+00,2,1545,?++??,False,428565698,1094024163,665458465,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
4,cg15580874,6:31529365_G_A,6:31377747_ACCGT_A,2,179315077,6,31529365,G,A,0.059,...,3.560000e-70,0.000000e+00,2,1545,?++??,False,428565698,1094071325,665505627,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
5,cg15580874,6:32042322_T_C,6:31098381_A_G,2,179315077,6,32042322,T,C,0.230,...,1.380000e-13,3.460000e-05,2,1742,++???,False,428565698,1094584282,666018584,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
6,cg15580874,6:32182759_G_A,6:32180241_C_T,2,179315077,6,32182759,G,A,0.137,...,2.010000e-45,0.000000e+00,2,1545,?++??,False,428565698,1094724719,666159021,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
7,cg15580874,6:32295744_T_A,6:31296388_G_C,2,179315077,6,32295744,T,A,0.494,...,6.160000e-65,0.000000e+00,2,1545,?++??,False,428565698,1094837704,666272006,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
8,cg15580874,6:32425225_CTT_CTTT,6:32585755_T_A,2,179315077,6,32425225,CTT,CTTT,0.436,...,2.020000e-154,0.000000e+00,2,1545,?++??,False,428565698,1094967185,666401487,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
9,cg15580874,6:32429277_A_G,6:32729734_C_G,2,179315077,6,32429277,A,G,0.252,...,2.030000e-58,0.000000e+00,2,1531,?-?-?,False,428565698,1094971237,666405539,DFNB59;PRKRA;MIR548N;PRKRA;PRKRA
